<a href="https://colab.research.google.com/github/abusufianrobin/PDF-QA-Bot/blob/main/PDF_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Install Dependencies

In [1]:
!pip install transformers torch pdfplumber sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

Step 2: Upload and Read the PDF
Now, upload the PDF file and extract its text:

In [2]:
import pdfplumber
from google.colab import files


uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]


def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_text = extract_text_from_pdf(pdf_filename)
print("Extracted text length:", len(pdf_text))


Saving CSE 4531 Project Proposal by 0112230721.pdf to CSE 4531 Project Proposal by 0112230721.pdf


Extracted text length: 4921


Step 3: Chunk the Text Properly
Since the model can't process huge text at once, we split it into smaller chunks (max 512 tokens).

In [3]:
import textwrap

# Split text into chunks
def chunk_text(text, max_length=512):
    return textwrap.wrap(text, max_length)

chunks = chunk_text(pdf_text, 512)  # Split into 512-character chunks
print(f"Total Chunks: {len(chunks)}")


Total Chunks: 10


Step 4: Load the flan-t5-large Model
Load the Hugging Face model for text summarization/QA:

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Step 5: Function to Answer Questions
We now define a function to answer user queries from the PDF.

In [5]:
import torch

def answer_question(question, text_chunks):
    best_answer = ""
    best_score = float("-inf")
    for chunk in text_chunks:

        input_text = f"question: {question} context: {chunk}"
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)


        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=100)

        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)


        if len(answer) > len(best_answer):
            best_answer = answer

    return best_answer


question = "What is the Methodology"
answer = answer_question(question, chunks)
print("Answer:", answer)


Answer: transaction data encrypted using HE-Transformer and CryptoNets. The server performs deep learning computations without decrypting the data, ensuring security. The fraud detection model follows these steps: i) Encrypted Input Processing: The server receives transaction data encrypted using HE-Transformer and CryptoNets. The server performs deep learning computations without decrypting the data, ensuring security. The fraud detection model follows these steps:


Step 6: Ask Questions Interactively
You can now ask multiple questions in a loop:

In [ ]:
while True:
    user_question = input("\nAsk a question (or type 'exit' to stop): ")
    if user_question.lower() == "exit":
        break
    response = answer_question(user_question, chunks)
    print("Answer:", response)



Ask a question (or type 'exit' to stop): What is the process of 1st step on methodology?
Answer: Research Project Title: Homomorphic encryption technology used for Credit Card fraud detection using deep learning (HE-Transformer, CryptoNets models)  Our Team Member: Name ID Mail

Ask a question (or type 'exit' to stop): Methodology 1st step?
Answer: Traditional fraud detection systems rely on machine learning and rule-based techniques to identify fraudulent transactions

Ask a question (or type 'exit' to stop): exit
